In [ ]:
from __future__ import print_function
%matplotlib nbagg

import datetime as dt
import os
import platform

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.dates import num2date, DateFormatter
from scipy.io import readsav
from sunpy.lightcurve import GOESLightCurve, LightCurve
from sunpy.time import TimeRange
from noaareport import NoaaReport

from utils import (
    onclick, posicao, calculo_de_indice, load_dados,
    calculo_da_media, CAMINHO_ABSOLUTO, ponto_mais_proximo
)

In [ ]:
dia = "18"
mes = "12"
ano = "2009"
df, time, diretorio = load_dados(dia, mes, ano)

In [ ]:
plt.rcParams['figure.figsize'] = 7,5
fig, axis = plt.subplots()

# Usa o index do dataframe para o eixo x.
axis.plot(df["L"], label="L")
axis.plot(df["R"], label="R")
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
plt.legend()
fig.canvas.mpl_connect('button_press_event', onclick)
fig.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" +"dia_inteiro")

In [ ]:
"""
 Ordem de seleção:
  Os dois últimos (últimos) selecionados são usados para o calculo das médias.
  Os dois penúltimos (primeiros) selecionados são usados para representar a média
  no grafico.
"""
dados_da_media = calculo_da_media(df)

indice_ig1 = dados_da_media[0]
indice_ig2 = dados_da_media[1]
medias = dados_da_media[2]
media_final_r = medias['R']
media_final_l = medias['L']
tempo1_flare = dados_da_media[3]
tempo2_flare = dados_da_media[4]

In [ ]:
fig_R, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["R"][tempo1_flare:tempo2_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

#plt.plot([df.index[tempo1_flare], df.index[tempo2_flare]],[media_final_r, media_final_r])

fig_L, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["L"][tempo1_flare:tempo2_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

#plt.plot([df.index[tempo1_flare], df.index[tempo2_flare]],[media_final_l, media_final_l])

In [ ]:
LM_normalizado = df["L_normalizado"][tempo1_flare:tempo2_flare]
RM_normalizado = df["R_normalizado"][tempo1_flare:tempo2_flare]

RL_V = df["R_normalizado"][tempo1_flare:tempo2_flare] - df["L_normalizado"][tempo1_flare:tempo2_flare]
RL_I = df["L_normalizado"][tempo1_flare:tempo2_flare] + df["R_normalizado"][tempo1_flare:tempo2_flare]

fig_IV, (axis, ax, ax2) = plt.subplots(3)

# Usa o mesmo index do dataframe para o eixo x.
axis.plot(LM_normalizado, label="L")
axis.plot(RM_normalizado, label="R")
axis.set_title("Analise 7 GHz")
axis.legend()
axis.set_ylabel("Flux (SFU)")
axis.yaxis.grid(color='gray')

ax.plot(RL_I)
ax.set_ylabel("I (SFU)")
ax.yaxis.grid(color='gray')

ax2.plot(RL_V)
ax2.set_ylabel("V (SFU)")
ax2.yaxis.grid(color='gray')

axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))
pico_7ghz = max(RL_I)

# Salva o gráfico.
nome_do_arquivo = "I-V-Flux(SFU)"
if platform.system() == "Linux":
    fig_IV.savefig(CAMINHO_ABSOLUTO + diretorio + "/" + nome_do_arquivo)
else:
    fig_IV.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + nome_do_arquivo)

In [ ]:
fig, axis = plt.subplots()

# Usa o index do dataframe para o eixo x.
axis.plot(df["R_normalizado"])
axis.plot(df["L_normalizado"])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
print(posicao[-1])
pico_7ghz = str(num2date(posicao[-1][1]))

## Região Ativa

In [ ]:
comeco_flare = str(df.index[tempo1_flare])[0:19]
fim_flare = str(df.index[tempo2_flare])[0:19]

caminho_eventos = "Eventos/" + ano + "_events/"
report = NoaaReport(ano, mes, dia, caminho_eventos)
noaa_df = report.get_dataframe()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(noaa_df)
ars = report.get_active_region(comeco_flare, fim_flare)

In [ ]:
ar = "None"
ar_noaa = "1035"

## Grau de polarização

In [ ]:
fig_7ghz, axis = plt.subplots()

# Usa o mesmo index do dataframe para o eixo x.
axis.plot(LM_normalizado, label="L")
axis.plot(RM_normalizado, label="R")
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

axis.legend()
axis.set_ylabel("Flux (SFU)")
axis.set_xlabel("Horário")
plt.title("Evento 7GHz")

fig_7ghz.canvas.mpl_connect('button_press_event', onclick)

filename = "grafico_normalizado"
if platform.system() == "Linux":
    fig_7ghz.savefig(CAMINHO_ABSOLUTO + diretorio + "/" + filename)
else:
    fig_7ghz.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + filename)

In [ ]:
posicao_inicial= num2date(posicao[-1][0])
indice_de_y1 = calculo_de_indice(df, posicao_inicial)

pico_R = max(df["R_normalizado"][tempo1_flare:tempo2_flare])
pico_L = max(df["L_normalizado"][tempo1_flare:tempo2_flare])

ponto_do_r_inicial = indice_de_y1[0][2]
ponto_do_r_inicial = float("{:.3f}".format(ponto_do_r_inicial))
print('Ponto do R inicial:', ponto_do_r_inicial)

ponto_do_r_no_pico = float(str(pico_R)[0:7])
ponto_do_r_no_pico = float("{:.3f}".format(ponto_do_r_no_pico))
print('Ponto do R no pico:', ponto_do_r_no_pico)

ponto_do_l_inicial = indice_de_y1[0][3]
ponto_do_l_inicial = float("{:.3f}".format(ponto_do_l_inicial))
print('Ponto do L incial:', ponto_do_l_inicial)

ponto_do_l_no_pico = float(str(pico_L)[0:7])
ponto_do_l_no_pico = float("{:.3f}".format(ponto_do_l_no_pico))
print('Ponto do L no pico:', ponto_do_l_no_pico)

# Procura o horario do pico.
for data in enumerate(df["R_normalizado"]):
    # Se encontrar o valor do pico, cria a variavel horario_do_pico.
    if data[1] == pico_R:
        horario_do_pico = df["R_normalizado"].index[data[0]]

tempo = str(horario_do_pico)[0:26]
print('Tempo:', tempo)

In [ ]:
I = ((ponto_do_r_no_pico - ponto_do_r_inicial) + (ponto_do_l_no_pico - ponto_do_l_inicial))
I = float("{:.3f}".format(I))
print("Cálculo do I:", I)

V = ((ponto_do_r_no_pico - ponto_do_r_inicial) - (ponto_do_l_no_pico - ponto_do_l_inicial))
V = float("{:.3f}".format(V))
print("Cálculo do V:", V)

In [ ]:
# Grau de polarização.
GP = (V / I) * 100
GP = float("{:.3f}".format(GP))
print('Grau de Polarização:', GP)

In [ ]:
horario_pico = str(horario_do_pico)[:19].replace(" ", "_")
comeco_flare = comeco_flare.replace(" ", "_")
fim_flare = fim_flare.replace(" ", "_")

In [ ]:
print("Começo: {}".format(comeco_flare))
print("Pico: {}".format(horario_pico))
print("Fim: {}".format(fim_flare))

In [ ]:
resultados_finais = np.transpose([
    [comeco_flare],
    [horario_pico],
    [fim_flare],
    [GP], [I], [V],
    [ar], [ar_noaa]
])
columns = ['begin','max','end', 'GP','I','V', 'AR', 'AR_NOAA']
dados = pd.DataFrame(resultados_finais, columns=columns)

if platform.system() == "Linux":
    dados.to_csv(CAMINHO_ABSOLUTO + "/eventos_wavelet/eventos.csv", header=None, sep='\t', mode='a', index=False)
else:
    dados.to_csv(CAMINHO_ABSOLUTO + "\eventos_wavelet\eventos.csv", header=None, sep='\t', mode='a', index=False)